In [21]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
import itertools
import re
from tokenizers import ByteLevelBPETokenizer
import os
from torch.nn.utils.rnn import pad_sequence
from functools import partial

In [2]:
FILE_PATH = 'data/en-pt.txt'
NUM_PHRASES = 1_000_000
OUTPUT_FILE = 'data/en-pt_sentences.txt'
TOKENIZER_DIR = 'tokenizer'
VOCAB_SIZE = 64_000

In [3]:
def load_dataset(dataset_path, limit=1000000):
    """
    Loads the dataset from the given path.

    Args:
        dataset_path: The path to the dataset file.
        limit: The maximum number of sentence pairs to load (default: 1000000).

    Returns:
        A list of (source, target) sentence pairs.
    """
    with open(dataset_path, "r", encoding="utf-8") as file:
        sentence_pairs = [tuple(line.strip().split("\t")) for line in itertools.islice(file, limit)]

    return sentence_pairs

## OLD-PREPROCESSING

In [41]:
def preprocess_text(text):
    """
    Preprocesses the given text.

    Args:
        text: The input text.
        remove_stopwords: Whether to remove stopwords from the text (default: True).

    Returns:
        A list of preprocessed words.
    """
    # Convert the text to lowercase
    text = text.lower()

    # Remove special characters and digits
    text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ]+', ' ', text)

    return text

In [42]:
def train_tokenizer(sentence_pairs,temp_sentences_path, vocab_size=VOCAB_SIZE, min_frequency=2, output_dir=TOKENIZER_DIR):
    """
    Trains a tokenizer on the given sentence pairs.

    Args:
        sentence_pairs: The list of (source, target) sentence pairs.
        vocab_size: The vocabulary size for the tokenizer.
        min_frequency: The minimum frequency for a token to be included in the vocabulary.
        output_dir: The directory to save the tokenizer files.
    """
    # Save all sentences to a temporary file
    with open(temp_sentences_path, "w", encoding="utf-8") as file:
        for src, tgt in sentence_pairs:
            file.write(src + "\n")
            file.write(tgt + "\n")

    # Train the tokenizer
    tokenizer = ByteLevelBPETokenizer()
    tokenizer.train(files=[temp_sentences_path], vocab_size=vocab_size, min_frequency=min_frequency, special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
        "<mask>",
    ])

    # Save the tokenizer
    tokenizer.save_model(output_dir)


In [43]:
class TranslationDataset(Dataset):
    def __init__(self, sentence_pairs, tokenizer, max_length):
        self.sentence_pairs = sentence_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentence_pairs)

    def __getitem__(self, idx):
        source, target = self.sentence_pairs[idx]
        tokenized_source = self.tokenizer.encode(source).ids[:self.max_length]
        tokenized_target = self.tokenizer.encode(target).ids[:self.max_length]
        return tokenized_source, tokenized_target

In [44]:
def load_tokenizer(tokenizer_dir=TOKENIZER_DIR):
    """
    Loads a tokenizer from the specified directory.

    Args:
        tokenizer_dir: The directory containing the tokenizer files.

    Returns:
        A Tokenizer object.
    """
    tokenizer = ByteLevelBPETokenizer(
        f"{tokenizer_dir}{os.sep}vocab.json",
        f"{tokenizer_dir}{os.sep}merges.txt",
        add_prefix_space=True
    )
    return tokenizer

In [45]:
def preprocess_data(sentence_pairs, tokenizer, max_length):
    """
    Preprocesses the sentence pairs by tokenizing and creating a TranslationDataset.

    Args:
        sentence_pairs: A list of (source, target) sentence pairs.
        tokenizer: The tokenizer used for tokenization.
        max_length: The maximum sequence length.

    Returns:
        A TranslationDataset object.
    """
    dataset = TranslationDataset(sentence_pairs, tokenizer, max_length)
    return dataset

In [46]:
def create_dataloader(dataset, batch_size,tokenizer, shuffle=True, num_workers=0):
    """
    Creates a DataLoader for the given dataset.

    Args:
        dataset: A PyTorch Dataset object.
        batch_size: The batch size to use in the DataLoader.
        shuffle: Whether to shuffle the dataset before creating the DataLoader.
        num_workers: The number of worker processes to use for loading the data.

    Returns:
        A DataLoader object.
    """
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        collate_fn=partial(collate_fn, tokenizer=tokenizer),
        pin_memory=True
    )

    return dataloader

def collate_fn(batch, tokenizer):
        src_tensors, tgt_tensors = zip(*batch)
        src_tensors = [torch.tensor(src) for src in src_tensors]
        tgt_tensors = [torch.tensor(tgt) for tgt in tgt_tensors]
        src_tensors = pad_sequence(src_tensors, batch_first=True, padding_value=tokenizer.token_to_id("<pad>"))
        tgt_tensors = pad_sequence(tgt_tensors, batch_first=True, padding_value=tokenizer.token_to_id("<pad>"))
        return src_tensors, tgt_tensors

In [32]:
# Load a smaller subset of the dataset
sentence_pairs = load_dataset(FILE_PATH, limit=NUM_PHRASES)

In [33]:
preprocessed_pairs = [(preprocess_text(en), preprocess_text(pt)) for en, pt in sentence_pairs]

In [54]:
for k, v in preprocessed_pairs:
    print(k)
    print(v)
    break

<s>  results for pain in midrand gauteng </s>
<s>  resultados encontrados para pain em midrand gauteng </s>


In [47]:
# Train and save the tokenizer
train_tokenizer(preprocessed_pairs, OUTPUT_FILE)

In [48]:
tokenizer = load_tokenizer()

In [12]:
tokenized_pairs = [(tokenizer.encode(en).ids, tokenizer.encode(pt).ids) for en, pt in preprocessed_pairs]

In [49]:
random.shuffle(preprocessed_pairs)
split_idx = int(len(preprocessed_pairs) * 0.9)  # 90% for training, 10% for validation
train_sentence_pairs = preprocessed_pairs[:split_idx]
val_sentence_pairs = preprocessed_pairs[split_idx:]

In [50]:
train_dataset = preprocess_data(train_sentence_pairs, tokenizer, 5000)
val_dataset = preprocess_data(val_sentence_pairs, tokenizer, 5000)

In [51]:
for k,v in train_dataset:
    print(k, v)
    break

[225, 32, 87, 34, 225, 2246, 355, 2733, 298, 61819, 12512, 272, 87, 34] [225, 32, 87, 34, 225, 2334, 7353, 353, 2733, 337, 61819, 12512, 272, 87, 34]


In [29]:
train_dataloader = create_dataloader(train_dataset, 1, tokenizer, shuffle=False, num_workers=0)
val_dataloader = create_dataloader(val_dataset, 1, tokenizer, shuffle=False, num_workers=0)

In [30]:
for k,v in train_dataloader:
    print(k, v)
    break

tensor([[  225,    45,    82,   552, 16383,   296, 12832,  3727, 10276, 11285,
          7498,  4254,   638,  8645,   297,  2441, 23252,    30,   296, 10077,
          9511,   539,  4489,  2313,   317,  8638,   310,    17,   265,  3088,
         35836,   297,  3727, 36647, 61299,    31,   296, 10077,  3356,   383,
         41298, 25165,   297, 12832,  3727, 10276, 35654,  1862,  8638,   310,
            17,   265,  3088,  9157,   324,    16,  8336,    31, 36289,   324,
          9009,  8428,   383,  1708,    17,   514, 56037, 13226, 11285, 22332,
           638,  5698,  3793,    18]]) tensor([[  225,    50,   831,  1473,   416,  4322,   545, 24734,  1182, 49270,
           282,  3892,   276, 15621,   354, 20581, 32263,    17,   475,   337,
          2511, 15925,    30,  3191,   371, 56289,   282, 14780,   282,  3312,
           600,   654,   337,  4246, 24000,   282,  3892,    31,  3191,   545,
         26518,   282, 19925, 62654, 30287,   465, 14780,   282,  3312,   600,
           65

# NEW-PREPROCESSING: Using Spacy Tokenizer